Goal: Implement a decision tree from scratch and apply it to the task of classifying whether a mushroom is edible or poisonous.

In [1]:
# Base packages

import numpy as np
import matplotlib.pyplot as plt
from public_tests import *
from utils import *

%matplotlib inline

We are playing the role of a company that grows and sells wild mushrooms. Nevermind the linguistics, apparently.

Anyway, poisonous mushrooms are a real concern, see? And we have some existing data to help us 
determind, based on physical traits, which ones are.

The dataset itself has $10$ examples with $3$ features ($x$):


1. Cap color (Browm, red)

2. Stalk shape (Tapering, enlargening)

3. Solitary (Yes, no)

And the output, ($y$):

Edible (Y/N)


For ease, we can one-hot encode these features to be:

1. Brown cap?

2. Tapering stalk shape?

3. Solitary

4. Edible

So a single example may look like:

$0101$

Indicating: Red cap, tapering, clustered, edible mushroom.


Time to load it in:

In [2]:
X_train = np.array([[1, 1, 1], [1, 0, 1], [1, 0, 0], [1, 0, 0], [1, 1, 1],
                   [0, 1, 1], [0, 0, 0], [1, 0, 1], [0, 1, 0], [1, 0, 0]])
y_train = np.array([1, 1, 0, 0, 1, 0, 0, 1, 1, 0])

In [5]:
# In a more complicated case, we may want to get a brief view of the data and the data type(s)
print(f"First 5 examples from X_train: \n{X_train[:5]}")
print(f"Type of X_train: {type(X_train)}")
print(f"\nFirst 5 results from y_train: {y_train[:5]}")
print(f"Type of y_train: {type(y_train)}")


First 5 examples from X_train: 
[[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Type of X_train: <class 'numpy.ndarray'>

First 5 results from y_train: [1 1 0 0 1]
Type of y_train: <class 'numpy.ndarray'>


In [8]:
# Another preliminary step is to check the dimensions of the data:
print(f"X_train's shape:\t\t\t{X_train.shape}")
print(f"y_train's shape:\t\t\t{y_train.shape}")
print(f"Number of training examples (m):\t{len(X_train)}")

X_train's shape:			(10, 3)
y_train's shape:			(10,)
Number of training examples (m):	10


As a refresher, building a decision tree has the following components:

1. Start with all examples in the root node.

2. Calculate information gain for splitting on all possible features ($3$), pick the feature with the highest information gain.

3. Split the dataset according to the selected feature, creating left and right branches of the tree.

4. Repeat this process until a stopping criterion is met.

$ $

To do these things, we need a couple of functions:
- Calculate node entropy
- Split dataset at a node into left/right branches based on a feature.
- Calculate information gain from splitting on a given feature.
- Choose feature that maximizes information gain.

(Plus some helper functions to repeat split until a criterion is met, in this case just a max depth of $2$).

Recall that entropy is calculated using $p_1$, the fraction of examples with $y=1$ of a subset as:

$$H(p_1) = -p_1log_2(p_1) - (1 - p_1)log_2(1 - p_1)$$

And for implementation purposes, $log_2(0) = 0$

Also remember to check the data at a node is not empty. Since our only criterion is depth $= 2$

If it is, return $0$

In [9]:
def compute_entropy(y):
    """
    Computes the entropy for a subset of data.
    
    Args:
        y (ndarray): Numpy array indicating whether each example at a node is edible (1) or not (0)
        
    Returns:
        entropy (float): Entropy at that node.
    """
    
    m = len(y)
    if m == 0:
        return 0
    
    p1 = np.sum(y) / m
    if p1 == 0 or p1 == 1:
        return 0
    
    return -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)


In [10]:
# 10 examples, 50/50 => Expect entropy = 1.0
print(f"Entropy at root: {compute_entropy(y_train)}")

Entropy at root: 1.0


In [12]:
def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into left and right branches.
    
    Args:
        X (ndarray): Data matrix of shape(n_samples, n_features)
        node_indices (list): List containing the active indices
        feature (int): Index of feature to split on
        
    Returns:
        left_indices (list): Indices with feature value == 1
        right_indices (list): Indices with feature value == 0
    """
    
    left_indices = []
    right_indices = []
    
    for i in node_indices:
        if X[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
            
    return left_indices, right_indices


In [15]:
root_indices = [x for x in range(10)]
feature = 0 # Cap color, brown = 1, red = 0
left_indices, right_indices = split_dataset(X_train, root_indices, feature)
print(left_indices, right_indices) # Expect right = [5, 6, 8], left is remaining

[0, 1, 2, 3, 4, 7, 9] [5, 6, 8]


Now we can split and calculate entropy, need to calculate information gain.

Recall that:

$$\text{Information Gain} = H(p_1^{\text{node}}) - (w^{\text{left}} H(p_1^{\text{left}}) + w^{\text{right}} H(p_1^{\text{right}}))$$

Where $w$ is the proportion of examples in each branch.

In [16]:
def compute_information_gain(X, y, node_indices, feature):
    """
    Computes the information gain from splitting a node on a given feature.
    
    Args:
        X (ndarray): Data matric of shape(n_samples, n_features)
        y (array like): list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices
        feature (int): Index of feature being split on
        
    Returns:
        information_gain (float): Information gain computer
    """
    
    # Initially, split the dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Helper variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # Compute information gain
    entropy_node = compute_entropy(y_node)
    entropy_left = compute_entropy(y_left)
    entropy_right = compute_entropy(y_right)
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    
    information_gain = entropy_node - (w_left * entropy_left + w_right * entropy_right)
    
    return information_gain

In [25]:
info_gains = [compute_information_gain(X_train, 
                                       y_train,
                                       [x for x in range(X_train.shape[0])],
                                       i)
              for i in range(X_train.shape[1])
             ]

In [33]:
info_gains # Max is on feature 2, is the sample is solitary or not. But let's automate this with a function.

[0.034851554559677034, 0.12451124978365313, 0.2780719051126377]

In [37]:
def get_best_split(X, y, node_indices):
    """
    Returns the optimal feature and threshold value to split the node data on.
    
    Args:
      X (ndarray): Data matrix of shape(n_samples, n_features)
      y (array like): list or ndarray with n_samples containing the target variable
      node_indices (ndarray): List containing the active indices
      
    Returns:
        best_feature (int): The index of the best feature to split on
    """
    
    n_samples, n_features = X.shape
    gains_per_feature = [compute_information_gain(X, y, node_indices, i) for i in range(n_features)]
    
    best_feature = np.argmax(gains_per_feature)
    
    if gains_per_feature[best_feature] == 0:
        return -1 # Pure set, no split is best...but really should be handled by criterion.
    
    return best_feature
    

In [40]:
get_best_split(X_train, y_train, root_indices) # Expect 2

2

Now, actually put this all together to build the tree.
We only have the one criterion, max depth of 2.

In [42]:
tree = []

def build_tree_recursion(X, y, node_indices, branch_name, curr_depth=0, max_depth=2):
    """
    Builds a tree using a recursive algorithm that splits the dataset into 2 subgroups at each node.
    Prints the tree.
    
    Args:
        X (ndarray): Data matrix of shape(n_samples, n_features)
        y (array like): list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices
        branch_name (string): Name of branch (Root, Left, or Right)
        max_depth (int): Max depth of resulting tree
        curr_depth (int): Current depth, used during recursion
    """
    
    # Base case - max depth reached
    if curr_depth == max_depth:
        formatting = " " * curr_depth + "-" * curr_depth
        print(f"{formatting} {branch_name} node with indices {node_indices}")
        return
    
    # If not base/final case, get best split, then act on it
    best_feat = get_best_split(X, y, node_indices)
    formatting = "-" * curr_depth
    print(f"{formatting} Depth {curr_depth}, {branch_name}: Split on feature {best_feat}")
    
    left, right = split_dataset(X, node_indices, best_feat)
    tree.append((left, right, best_feat))
    
    # Keep the recursion going
    build_tree_recursion(X, y, left, "Left", curr_depth + 1)
    build_tree_recursion(X, y, right, "Right", curr_depth + 1)
    

In [44]:
build_tree_recursion(X_train, y_train, root_indices, "Root")

 Depth 0, Root: Split on feature 2
- Depth 1, Left: Split on feature 0
  -- Left node with indices [0, 1, 4, 7]
  -- Right node with indices [5]
- Depth 1, Right: Split on feature 1
  -- Left node with indices [8]
  -- Right node with indices [2, 3, 6, 9]
